In [ ]:
import numpy as np
import tkinter as tk
from tkinter import messagebox
import torch
from game import Game
from agent import Agent, User
from model1 import Residual_CNN
import config

class GomokuGUI:
    def __init__(self, master, model_path, device):
        self.master = master
        self.master.title("Gomoku Game")

        # 게임 환경 초기화
        self.env = Game()
        self.board_size = int(self.env.state_size ** 0.5)
        self.device = device

        # 모델 로드
        self.model = Residual_CNN(
            config.REG_CONST,
            config.LEARNING_RATE,
            self.env.input_shape,
            self.env.action_size,
            config.HIDDEN_CNN_LAYERS,
        )
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.to(self.device)

        # 플레이어 초기화
        self.user = User("User", self.env.state_size, self.env.action_size)
        self.ai = Agent(
            "AI", self.env.state_size, self.env.action_size, config.MCTS_SIMS, config.CPUCT, self.model, device=self.device
        )

        self.canvas = tk.Canvas(
            master, width=self.board_size * 40, height=self.board_size * 40, bg="white"
        )
        self.canvas.pack()

        # 바둑판 그리기
        self.draw_board()

        # 이벤트 바인딩
        self.canvas.bind("<Button-1>", self.handle_click)

        # 게임 상태 초기화
        self.current_player = 1  # 1은 유저, -1은 AI

    def draw_board(self):
        """바둑판 그리기"""
        for i in range(self.board_size):
            self.canvas.create_line(20, 20 + i * 40, 20 + (self.board_size - 1) * 40, 20 + i * 40)
            self.canvas.create_line(20 + i * 40, 20, 20 + i * 40, 20 + (self.board_size - 1) * 40)

    def draw_stone(self, row, col, player):
        """돌 그리기"""
        color = "black" if player == 1 else "white"
        x0, y0 = 20 + col * 40 - 15, 20 + row * 40 - 15
        x1, y1 = 20 + col * 40 + 15, 20 + row * 40 + 15
        self.canvas.create_oval(x0, y0, x1, y1, fill=color)

    def handle_click(self, event):
        """유저 클릭 처리"""
        col = (event.x - 20) // 40
        row = (event.y - 20) // 40

        if col < 0 or col >= self.board_size or row < 0 or row >= self.board_size:
            return

        action = row * self.board_size + col

        if self.env.gameState.board[action] != 0:
            messagebox.showwarning("Invalid Move", "This cell is already occupied!")
            return

        # 유저 돌 두기
        self.env.step(action)
        self.draw_stone(row, col, 1)

        if self.env.gameState.isTerminal():
            messagebox.showinfo("Game Over", "You win!")
            self.canvas.unbind("<Button-1>")
            return

        # AI 차례
        self.master.after(500, self.ai_turn)

    def ai_turn(self):
        """AI의 턴"""
        ai_action, _, _, _ = self.ai.act(self.env.gameState, tau=0)
        row, col = divmod(ai_action, self.board_size)

        self.env.step(ai_action)
        self.draw_stone(row, col, -1)

        if self.env.gameState.isTerminal():
            messagebox.showinfo("Game Over", "AI wins!")
            self.canvas.unbind("<Button-1>")

if __name__ == "__main__":
    # 모델 경로 및 디바이스 설정
    model_path = "/home/aikusrv01/omok/torch_omok_test/DeepReinforcementLearning/models/gomokucurr_0025.pt"
    device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

    # Tkinter 초기화
    root = tk.Tk()
    app = GomokuGUI(root, model_path, device)
    root.mainloop()


TclError: no display name and no $DISPLAY environment variable